**Preload IOMP before start the Notebook**

In [1]:
import torch
import torch.fx.experimental.optimization as optimization
import intel_extension_for_pytorch as ipex

In [2]:
# Prepare the simple convolution network
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv = torch.nn.Conv2d(64, 128, (3, 3), stride=(2, 2), padding=(1, 1), bias=False)

    def forward(self, x):
        x1 = self.conv(x)
        y = torch.flatten(x1, start_dim=1)
        return y
model = SimpleNet().eval()
x = torch.rand(64, 64, 3, 3)

In [3]:
# Thread Affinity API Experiments
cpu_pool = ipex.cpu.runtime.CPUPool([1, 2, 3, 4])
with ipex.cpu.runtime.pin(cpu_pool):
    for i in range(2000):
        y_runtime = model(x)

In [4]:
# Async Task API Experiments
cpu_pool1 = ipex.cpu.runtime.CPUPool([1, 2, 3])
cpu_pool2 = ipex.cpu.runtime.CPUPool([6, 7, 8])

task1 = ipex.cpu.runtime.Task(model, cpu_pool1)
task2 = ipex.cpu.runtime.Task(model, cpu_pool2)

y1_future = task1(x)
y2_future = task2(x)

y1 = y1_future.get()
y2 = y2_future.get()